Create dummy data with:
- `device_id`: 10 different devices
- `record_id`: 10k unique records
- `feature_1`: a feature for model training
- `feature_2`: a feature for model training
- `feature_3`: a feature for model training
- `label`: the variable we're trying to predict

In [6]:
import pyspark.sql.functions as f

df = (spark.range(100*100)
  .select(f.col("id").alias("record_id"), (f.col("id") % 10).alias("device_id"))
  .withColumn("feature_1", f.rand() * 1)
  .withColumn("feature_2", f.rand() * 2)
  .withColumn("feature_3", f.rand() * 3)
  .withColumn("label", (f.col("feature_1") + f.col("feature_2") + f.col("feature_3")) + f.rand())
)

df.show()

Command took 0.20 seconds -- by niall.turbitt@databricks.com at 02/13/2020, 04:13:07 PM UTC on Jupyter

Accordion(children=(VBox(),), layout=Layout(display='none'), selected_index=None)

+---------+---------+--------------------+--------------------+-------------------+------------------+
|record_id|device_id|           feature_1|           feature_2|          feature_3|             label|
+---------+---------+--------------------+--------------------+-------------------+------------------+
|        0|        0|  0.3798100624574545|  1.4202490062082653| 1.6781901282345544| 3.525612733862513|
|        1|        1|  0.9107536883090278|   1.442838269067278|  2.611383485089366| 5.814245271889851|
|        2|        2| 0.41590416052434676| 0.10418460693028986|0.22604878358315217|1.6008841116808505|
|        3|        3|  0.5512047715185051|  1.8502137667712624| 2.7280610877991984| 5.679620353053467|
|        4|        4|  0.6497210433084482|  1.4206298832076816| 0.7105026181927976| 3.736893714115734|
|        5|        5|  0.9773319027898405| 0.12361450125920048| 1.3702032077472008|2.7353747472986503|
|        6|        6|  0.4380566234523088|  1.3427775095169203|  1.073934

/databricks/python3/lib/python3.7/site-packages/ipywidgets/widgets/widget.py:410: DeprecationWarning: Passing unrecoginized arguments to super(IntProgress).__init__(step=1).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Enable Apache Arrow

In [7]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Command took 0.01 seconds -- by niall.turbitt@databricks.com at 02/13/2020, 04:15:32 PM UTC on Jupyter

Accordion(children=(VBox(),), layout=Layout(display='none'), selected_index=None)

Define the return schema

In [8]:
import pyspark.sql.types as t

trainReturnSchema = t.StructType([
  t.StructField('device_id', t.IntegerType()),    # unique device ID
  t.StructField('n_used', t.IntegerType()),       # number of records used in training
  t.StructField('model_path', t.StringType()),    # path to the model for a given device
  t.StructField('mse', t.FloatType())             # metric for model performance
])

Command took 0.01 seconds -- by niall.turbitt@databricks.com at 02/13/2020, 04:15:34 PM UTC on Jupyter

Accordion(children=(VBox(),), layout=Layout(display='none'), selected_index=None)

Define a pandas UDF that takes all the data for a given device, train a model, saves it as a nested run, and returns a spark object with the above schema

In [11]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

@f.pandas_udf(trainReturnSchema, functionType=f.PandasUDFType.GROUPED_MAP)
def train_model(df_pandas):
    """
    Trains an sklearn model on grouped instances
    """
    # Pull metadata
    device_id = df_pandas['device_id'].iloc[0]
    n_used = df_pandas.shape[0]
    run_id = df_pandas['run_id'].iloc[0]
  
    # Train the model
    input_columns = ['feature_1', 'feature_2', 'feature_3']
    X = df_pandas[input_columns]
    y = df_pandas['label']
    rf = RandomForestRegressor()
    rf.fit(X, y)

    # Evaluate the model
    predictions = rf.predict(X)
    mse = mean_squared_error(y, predictions) # NOTE NO TRAIN/TEST SPLIT

    # Log the results as a nested run
    # Note that we need 2 with blocks due to the distributed nature of this action
    with mlflow.start_run(run_id=run_id):
        with mlflow.start_run(run_name=str(device_id), nested=True) as run:
            mlflow.sklearn.log_model(rf, str(device_id))
            mlflow.log_metric("mse", mse)

            artifact_uri = run.info.artifact_uri + "/" + str(device_id)
            # Create a return pandas DataFrame that matches the schema above
            returnDF = pd.DataFrame([[device_id, n_used, artifact_uri, mse]], 
            columns=["device_id", "n_used", "model_path", "mse"])

    return returnDF 


Command took 0.30 seconds -- by niall.turbitt@databricks.com at 02/13/2020, 04:17:33 PM UTC on Jupyter

Accordion(children=(VBox(),), layout=Layout(display='none'), selected_index=None)

In [15]:
with mlflow.start_run(run_name="Parent Run") as run:
    run_id = run.info.run_uuid
    
    print(run_id)
    

Command took 0.25 seconds -- by niall.turbitt@databricks.com at 02/13/2020, 04:24:02 PM UTC on Jupyter

Accordion(children=(VBox(),), layout=Layout(display='none'), selected_index=None)

4aa97073258b4726bb1381f3ce713a2a


Apply the pandas UDF to grouped data

In [17]:
import tempfile

mlflow.set_experiment(f"/Users/niall.turbitt@databricks/test_workspace_2")

# Create the parent run and add run_id to the workers so they can find the parent run
with mlflow.start_run(run_name="Parent Run") as run:

    run_id = run.info.run_uuid
  
  # This is doing most of the work
    modelDirectoriesDF = df.withColumn("run_id", f.lit(run_id)).groupby("device_id").apply(train_model)
  
  # Log modelDirectoriesDF to the parent run using a temporary file
    temp = tempfile.NamedTemporaryFile(prefix="modelDirectoriesDF-", suffix=".csv")
    temp_name = temp.name
    try:
        modelDirectoriesDF.toPandas().to_csv(temp_name, index=False)
        mlflow.log_artifact(temp_name, "modelDirectoriesDF.csv")
    finally:
        temp.close() # Delete the temp file
        
modelDirectoriesDF.show()

Command took 10.28 seconds -- by niall.turbitt@databricks.com at 02/13/2020, 04:36:21 PM UTC on Jupyter

Accordion(children=(VBox(),), layout=Layout(display='none'), selected_index=None)

INFO: '/Users/niall.turbitt@databricks/test_workspace_2' does not exist. Creating a new experiment


/databricks/python3/lib/python3.7/site-packages/ipywidgets/widgets/widget.py:410: DeprecationWarning: Passing unrecoginized arguments to super(IntProgress).__init__(step=1).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
/databricks/spark/python/pyspark/sql/dataframe.py:2195: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o1769.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:358)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala

Py4JJavaError: An error occurred while calling o1769.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:358)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:67)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:63)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection

Combine the orignal data to the new DataFrame so we can use `model_path`

In [13]:
combinedDF = (df
  .join(modelDirectoriesDF, on="device_id", how="left")
)

combinedDF.show()

Command took 10.54 seconds -- by niall.turbitt@databricks.com at 02/13/2020, 04:20:07 PM UTC on Jupyter

Accordion(children=(VBox(),), layout=Layout(display='none'), selected_index=None)

Py4JJavaError: An error occurred while calling o1465.showString.
: org.apache.spark.SparkException: Exception thrown in Future.get: 
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:195)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:391)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:168)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:156)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:188)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:184)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:156)
	at org.apache.spark.sql.execution.joins.BroadcastHashJoinExec.prepareBroadcast(BroadcastHashJoinExec.scala:

Define a pandas UDF to apply the model.  **Note this only needs 1 read from DBFS per device**

In [14]:
applyReturnSchema = t.StructType([
  t.StructField('record_id', t.IntegerType()),
  t.StructField('device_id', t.LongType()),
  t.StructField('prediction', t.FloatType())
])

@f.pandas_udf(applyReturnSchema, functionType=f.PandasUDFType.GROUPED_MAP)
def apply_model(df_pandas):
  '''
  Applies model
  '''
  device_id = df_pandas['device_id'].iloc[0]
  model_path = df_pandas['model_path'].iloc[0]
  
  input_columns = ['feature_1', 'feature_2', 'feature_3']
  X = df_pandas[input_columns]
  
  model = mlflow.sklearn.load_model(model_path)
  prediction = model.predict(X)
  
  returnDF = pd.DataFrame({
    "record_id": df_pandas['record_id'],
    "prediction": prediction
  })
  returnDF["device_id"] = device_id

  return returnDF

predictionDF = combinedDF.groupby("device_id").apply(apply_model)
display(predictionDF)

record_id,device_id,prediction
2230,0,2.7645466
2240,0,3.4732797
2250,0,1.1655287
2260,0,1.8476894
2270,0,4.6743846
2280,0,2.6057968
2290,0,3.1509695
2300,0,3.2745278
2310,0,3.583009
2320,0,2.7822325


You made it to the end!  Here's a picture of a unicorn:

![unicorn](https://www.jing.fm/clipimg/detail/37-375094_galaxy-unicorn-png.png)